# Sentiment classification with LSTM
In this notebook we will use LSTMs to do sentiment classification on the [imdb dataset](http://ai.stanford.edu/~amaas/data/sentiment/). 

In [2]:
import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 

## Dataset

To get the data: <br>
`wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz`

In [4]:
def unpack_dataset():
    ! mkdir -p data/aclImdb
    ! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    ! tar -zxvf aclImdb_v1.tar.gz -C data

In [6]:
#unpack_dataset()

In [3]:
from pathlib import Path
PATH = Path("data/aclImdb/")
list(PATH.iterdir())

[PosixPath('data/aclImdb/imdbEr.txt'),
 PosixPath('data/aclImdb/test'),
 PosixPath('data/aclImdb/imdb.vocab'),
 PosixPath('data/aclImdb/README'),
 PosixPath('data/aclImdb/train')]

In [4]:
path = PATH/"train/pos/0_9.txt"
path.read_text()

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

## Tokenization

In [5]:
# first time run this
#!python3 -m spacy download en

In [6]:
re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)

my_tok = spacy.load('en')
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(sub_br(x))]

In [7]:
path = PATH/"train/pos/0_9.txt"
spacy_tok(path.read_text())[:10]

['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at']

### Computing vocab2index

In [8]:
pos_files = list((PATH/"train"/"pos").iterdir())
neg_files = list((PATH/"train"/"neg").iterdir())
all_files = pos_files + neg_files
all_files[:5]

[PosixPath('data/aclImdb/train/pos/4715_9.txt'),
 PosixPath('data/aclImdb/train/pos/12390_8.txt'),
 PosixPath('data/aclImdb/train/pos/8329_7.txt'),
 PosixPath('data/aclImdb/train/pos/9063_8.txt'),
 PosixPath('data/aclImdb/train/pos/3092_10.txt')]

In [9]:
counts = Counter()
for path in all_files:
    counts.update(spacy_tok(path.read_text()))

In [28]:
#counts

In [29]:
len(counts.keys())

33914

In [30]:
for word in list(counts):
    if counts[word] < 5:
        del counts[word]

In [31]:
len(counts.keys())

33914

In [32]:
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [33]:
#vocab2index

## Dataset

In [34]:
# note that spacy_tok takes a while run it just once
def encode_sentence(path, vocab2index, N=400, padding_start=True):
    x = spacy_tok(path.read_text())
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in x])
    l = min(N, len(enc1))
    if padding_start:
        enc[:l] = enc1[:l]
    else:
        enc[N-l:] = enc1[:l]
    return enc, l

In [35]:
path = PATH/"train/neg/211_4.txt"
encode_sentence(path, vocab2index, N=400, padding_start=False)

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            1,  6069,    40,  3675,    40,   391,     3,  3812,    13,
         1828,  3517,    58,   185,   822,    63,   205,    69,   525,
          731,   132,  2250, 16153,   132,  9762,    84,   497,  4027,
           84, 26204,   115,  2101,    71,  2066,     3,     1,   252,
      

In [36]:
class ImdbDataset(Dataset):
    def __init__(self, PATH, train="train", N=400, padding_start=True):
        self.path_to_images = PATH/train
        self.pos_files = list((self.path_to_images/"pos").iterdir())
        self.neg_files = list((self.path_to_images/"neg").iterdir())
        self.files = self.pos_files + self.neg_files
        # pos 1, neg 0
        self.y = np.concatenate((np.ones(len(self.pos_files), dtype=int),
                                np.zeros(len(self.neg_files), dtype=int)), axis=0)
        # it is important to run encode_sentence in the init
        self.X = [encode_sentence(path, vocab2index, N, padding_start) for path in self.files]
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x, s = self.X[idx]
        return x, s, self.y[idx]

In [37]:
train_ds_v0 = ImdbDataset(PATH, padding_start=False)
valid_ds_v0 = ImdbDataset(PATH, "test", padding_start=False)

In [38]:
batch_size = 1000
train_dl_v0 = DataLoader(train_ds_v0, batch_size=batch_size, shuffle=True)
valid_dl_v0 = DataLoader(valid_ds_v0, batch_size=batch_size)

In [39]:
train_ds_v0[1]

(array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

## Understanding LSTMs 

In [40]:
# Input dim is the dimension of the embedding for each word (2 in the example)
# Output dim is the dimension of the hidden layer (4 in this example)
# batch_first – If True, then the input and output tensors are provided as (batch, seq, feature). 
lstm = nn.LSTM(2, 4, batch_first=True)  

In [41]:
inputs = [torch.randn(1, 2) for _ in range(5)] # make a sequence of length 5
inputs = torch.cat(inputs).view(1, len(inputs), -1)
inputs

tensor([[[-0.5128,  1.4743],
         [-1.4117, -1.0552],
         [-2.1035,  0.2079],
         [-0.0191, -1.2118],
         [-0.2157,  1.2554]]])

In [42]:
# RNNs with batch_first=True assume this input shape
# input shape should be bash_size x seq_len x embedding dimension
inputs.shape

torch.Size([1, 5, 2])

In [43]:
out, (hidden, cell) = lstm(inputs)

In [44]:
print(out.shape)
out

torch.Size([1, 5, 4])


tensor([[[ 0.0362,  0.3095, -0.2179,  0.0650],
         [ 0.1125, -0.0461, -0.3170, -0.0404],
         [ 0.0267, -0.0979, -0.3857, -0.0164],
         [ 0.1729, -0.0888, -0.2652, -0.0342],
         [ 0.0649,  0.2441, -0.3059,  0.0615]]], grad_fn=<TransposeBackward0>)

In [45]:
hidden

tensor([[[ 0.0649,  0.2441, -0.3059,  0.0615]]], grad_fn=<StackBackward>)

## LSTM V0 model

In [46]:
class LSTMV0Model(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(LSTMV0Model,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        out_pack, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [47]:
def train_epocs_v0(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            # s is not used in this model
            x = x.long() #.cuda()
            y = y.float() #.cuda()
            y_pred = model(x)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics_v0(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [48]:
def val_metrics_v0(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in valid_dl:
        # s is not used here
        x = x.long() #.cuda()
        y = y.float().unsqueeze(1) #.cuda()
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [49]:
batch_size = 5000
train_dl = DataLoader(train_ds_v0, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds_v0, batch_size=batch_size)

In [50]:
vocab_size = len(words)
print(vocab_size)
model_v0 = LSTMV0Model(vocab_size, 50, 50) #.cuda()

33916


In [51]:
train_epocs_v0(model_v0, epochs=30, lr=0.01)

train loss 0.688 val loss 0.683 and val accuracy 0.542
train loss 0.637 val loss 0.629 and val accuracy 0.640
train loss 0.542 val loss 0.561 and val accuracy 0.735
train loss 0.428 val loss 0.511 and val accuracy 0.783
train loss 0.370 val loss 0.619 and val accuracy 0.751
train loss 0.310 val loss 0.507 and val accuracy 0.820


In [52]:
train_epocs_v0(model_v0, epochs=30, lr=0.001)

train loss 0.255 val loss 0.544 and val accuracy 0.827
train loss 0.243 val loss 0.554 and val accuracy 0.827
train loss 0.241 val loss 0.551 and val accuracy 0.830
train loss 0.233 val loss 0.550 and val accuracy 0.833
train loss 0.228 val loss 0.582 and val accuracy 0.830
train loss 0.219 val loss 0.572 and val accuracy 0.834


## LSTM model with variable length

In [53]:
# dataset with padding at the end
train_ds = ImdbDataset(PATH)
valid_ds = ImdbDataset(PATH, "test")

### Debugging our model

In [54]:
batch_size = 7
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

x,s,y = next(iter(train_dl)) # here s is the length of the sentences

In [55]:
x.shape, s.shape

(torch.Size([7, 400]), torch.Size([7]))

In [56]:
s

tensor([176,  96, 204, 232, 149, 166, 175])

In [57]:
y

tensor([0, 0, 0, 1, 1, 1, 0])

In [58]:
# sort by length so we can use pack_padded_sequence
s, index = s.sort(0, descending=True)
x = x[index]

In [59]:
s

tensor([232, 204, 176, 175, 166, 149,  96])

In [60]:
index

tensor([3, 2, 0, 6, 5, 4, 1])

In [61]:
y[index]

tensor([1, 0, 0, 0, 1, 1, 0])

In [62]:
vocab_size = len(words)
embedding_dim = 10
embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

In [63]:
x = embed(x.long())
x.shape

torch.Size([7, 400, 10])

In [64]:
hidden_dim = 9
lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

In [65]:
# RNN will not perform calculation on pad elements if pack_padded_sequence is used
x_pack = pack_padded_sequence(x, list(s), batch_first=True)

In [66]:
out_pack, (ht, ct) = lstm(x_pack)

In [67]:
## final hidden layer
ht.shape

torch.Size([1, 7, 9])

In [68]:
ht[-1].shape

torch.Size([7, 9])

In [69]:
linear = nn.Linear(hidden_dim, 1)
y_hat = linear(ht[-1])
y_hat

tensor([[0.2167],
        [0.2753],
        [0.2194],
        [0.1846],
        [0.1855],
        [0.1711],
        [0.1631]], grad_fn=<AddmmBackward>)

In [70]:
index.unsqueeze(1).shape

torch.Size([7, 1])

In [71]:
# takes back to the original ordering
h = torch.zeros_like(y_hat).scatter_(0, index.unsqueeze(1), y_hat)

In [72]:
h

tensor([[0.2194],
        [0.1631],
        [0.2753],
        [0.2167],
        [0.1711],
        [0.1855],
        [0.1846]], grad_fn=<ScatterBackward0>)

In [73]:
y_hat

tensor([[0.2167],
        [0.2753],
        [0.2194],
        [0.1846],
        [0.1855],
        [0.1711],
        [0.1631]], grad_fn=<AddmmBackward>)

In [74]:
index

tensor([3, 2, 0, 6, 5, 4, 1])

### Model v1

In [88]:
class LSTMModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(LSTMModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.5)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        
    def forward(self, x, s):
        # sorting
        s, sort_index = torch.sort(s, 0,descending=True)
        s = s.numpy().tolist()
        x = x[sort_index]
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = self.linear(ht[-1])
        return torch.zeros_like(out).scatter_(0, sort_index.unsqueeze(1), out) #.cuda()

In [89]:
def train_epocs(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.long() #.cuda()
            y = y.float() #.cuda()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        if i % 5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [90]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in valid_dl:
        x = x.long()  #.cuda()
        y = y.float().unsqueeze(1) #.cuda()
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [ ]:
batch_size = 2000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [ ]:
vocab_size = len(words)
print(vocab_size)
model = LSTMModel(vocab_size, 50, 50) #.cuda()

In [ ]:
train_epocs(model, epochs=30, lr=0.01)

train loss 0.681 val loss 0.664 and val accuracy 0.593


In [ ]:
train_epocs(model, epochs=30, lr=0.001)

In [290]:
train_epocs(model, epochs=30, lr=0.001)

train loss 0.222 val loss 0.611 and val accuracy 0.814
train loss 0.204 val loss 0.677 and val accuracy 0.810
train loss 0.207 val loss 0.666 and val accuracy 0.812
train loss 0.197 val loss 0.625 and val accuracy 0.822
train loss 0.190 val loss 0.695 and val accuracy 0.813
train loss 0.188 val loss 0.696 and val accuracy 0.815


In [222]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

In [223]:
p = PATH/"models/model-81.pth"
save_model(model, p)

In [224]:
val_metrics(model, valid_dl)

(0.6520548796653748, tensor(0.8236, device='cuda:0'))

In [225]:
load_model(model, p)

## GRU model with dropout

In [297]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(GRUModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(0.5)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        
    def forward(self, x, s):
        s, sort_index = torch.sort(s, 0,descending=True)
        s = s.numpy().tolist()
        x = x[sort_index]
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, list(s), batch_first=True)
        out_pack, ht= self.gru(x_pack)
        out = self.linear(ht[-1])
        return torch.zeros_like(out).scatter_(0, sort_index.unsqueeze(1).cuda(), out)

In [298]:
vocab_size = len(words)
print(vocab_size)
model2 = GRUModel(vocab_size, 50, 50).cuda()

33920


In [299]:
train_epocs(model2, epochs=30, lr=0.01)

train loss 0.687 val loss 0.682 and val accuracy 0.554
train loss 0.582 val loss 0.610 and val accuracy 0.712
train loss 0.404 val loss 0.590 and val accuracy 0.762
train loss 0.279 val loss 0.416 and val accuracy 0.852
train loss 0.182 val loss 0.491 and val accuracy 0.861
train loss 0.141 val loss 0.496 and val accuracy 0.869


In [300]:
p = PATH/"models/model-gru-87.pth"
save_model(model2, p)

## Bidirectional and multiple layers GRUs / LSTMs

In [301]:
batch_size = 7
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

x,s,y = next(iter(train_dl)) # here s is the length of the sentences

In [302]:
vocab_size = len(words)
embedding_dim = 10
hidden_dim = 9
embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
lstm1 = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
lstm2 = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)

In [303]:
s, index = s.sort(0, descending=True)
x = x[index]
x = embed(x.long())
x_pack = pack_padded_sequence(x, list(s), batch_first=True)

In [304]:
lstm_out, (ht, ct) = lstm1(x)

In [305]:
ht.shape

torch.Size([2, 7, 9])

In [306]:
ht[-2,:,:].shape

torch.Size([7, 9])

In [307]:
lstm_out, (ht2, ct2) = lstm2(x)

In [308]:
ht2.shape

torch.Size([4, 7, 9])

In [309]:
ht2[-2,:,:].shape, ht2[-1,:,:].shape

(torch.Size([7, 9]), torch.Size([7, 9]))

In [310]:
#concat the final forward (ht[-2,:,:]) and backward (ht[-1,:,:]) hidden layers      
h = torch.cat((ht2[-2,:,:], ht2[-1,:,:]), dim = 1)
h.shape

torch.Size([7, 18])

In [311]:
# This is slow

In [312]:
class LSTMBiModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(LSTMBiModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True,
                            dropout=0.3, bidirectional=True)
        self.linear = nn.Linear(2*hidden_dim, 1)
        
    def forward(self, x, s):
        s, sort_index = torch.sort(s, 0,descending=True)
        s = s.numpy().tolist()
        x = x[sort_index]
        x = self.embeddings(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True)
        out_pack, (ht, ct) = self.lstm(x_pack)
        h = torch.cat((ht[-2,:,:], ht[-1,:,:]), dim = 1)
        h = self.linear(h)
        return torch.zeros_like(h).scatter_(0, sort_index.unsqueeze(1).cuda(), h)

In [313]:
vocab_size = len(words)
model3 = LSTMBiModel(vocab_size, 50, 50).cuda()

In [ ]:
train_epocs(model3, epochs=15, lr=0.01)

## Bi GRUS

In [ ]:
class GRUBiModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(GRUBiModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True,
                            dropout=0.3, bidirectional=True)
        self.linear = nn.Linear(2*hidden_dim, 1)
        
    def forward(self, x, s):
        x = self.embeddings(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True)
        out_pack, ht = self.gru(x_pack)
        h = torch.cat((ht[-2,:,:], ht[-1,:,:]), dim = 1)
        return self.linear(h)

## Exercise:
Start with pre-trained embeddings.

## References

The model in this notebook is adapted from this [pytorch tutorial](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html). 